# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [16]:
import matplotlib
matplotlib.use("TkAgg")
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import sympy as sp

from PIL import Image
import pickle

# Provided functions

In [3]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least {} lines to compute vanishing point'.format(min_lines))
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least {} lines, you have {} now'.format(min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [4]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [10]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR CODE>
#     pt1 = np.cross(lines[:, 0], lines[:, 1])/np.cross(lines[:, 0], lines[:, 1])[-1]
#     pt2 = np.cross(lines[:, 1], lines[:, 2])/np.cross(lines[:, 1], lines[:, 2])[-1]
#     pt3 = np.cross(lines[:, 0], lines[:, 2])/np.cross(lines[:, 0], lines[:, 2])[-1]
#     point = (pt1 + pt2 + pt3)/3
#     return point
    second_moment = lines.dot(lines.T)
    w, v = np.linalg.eig(second_moment)
    # Find min eigenvalue and eigen vec.
    min_idx = np.argmin(w)
    vp = v[:, min_idx]
    # Convert to homogeneous.
    vp = vp / vp[-1]
    return vp


In [6]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR CODE>
    line = np.cross(vpts[:,0], vpts[:,2])
    line = line/np.sqrt(line[0]**2 + line[1]**2)
    return line

In [7]:
def plot_horizon_line(im, vpts):#im, line):
    """
    Plots the horizon line.
    """
    pt1, pt2 = vpts[:, 0], vpts[:, 2]
    plt.imshow(im)
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')
    plt.plot(pt1[0], pt1[1], 'ro')
    plt.plot(pt2[0], pt2[1], 'ro')
    plt.title('Horizon Line')
    plt.show()
    return
    

In [32]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR CODE>
    f, u, v = sp.Symbol('f'),sp.Symbol('u'),sp.Symbol('v')
    
    K = sp.Matrix(((f, 0, u), (0, f, v), (0, 0, 1))).inv()
    e1 = sp.Matrix(vpts[:, 0]).T * K.T * K* sp.Matrix(vpts[:, 1])
    e2 = sp.Matrix(vpts[:, 0]).T * K.T * K* sp.Matrix(vpts[:, 2])
    e3 = sp.Matrix(vpts[:, 1]).T * K.T * K* sp.Matrix(vpts[:, 2])
    
    sol = sp.solve([e1, e2, e3], [f, u, v])[0]
    K = sp.Matrix(((sol[0],0,sol[1]),(0,sol[0],sol[2]), (0,0,1)))
    return sol[0], sol[1], sol[2], K 

In [54]:
def get_rotation_matrix(K, vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    
    a = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 1])
    b = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 2])
    c = np.matmul(np.array(K.inv()).astype(np.float), vpts[:, 0])
    a = (a/np.linalg.norm(a)).T.reshape(3,1)
    b = (b/np.linalg.norm(b)).T.reshape(3,1)
    c = (c/np.linalg.norm(c)).T.reshape(3,1)
    print(a,b)
    R = np.concatenate((a,b,c)) 
    return R

In [ ]:
def get_homography(...):
    """
    Compute homography for transforming the image into fronto-parallel 
    views along the different axes.
    """
    # <YOUR CODE>
    pass

In [ ]:
def get_rotation_matrix_rectification(...):
    """
    Compute the rotation matrix that will be used to compute the 
    homography for rectification.
    """
    # <YOUR CODE>
    pass

# Main function

In [8]:
im = np.asarray(Image.open('./data/Q3/eceb.png'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('data.pickle'):
    with open('data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f)

In [8]:
# Click and save the line data for vanishing points. This snippet 
# opens up an interface for selecting points and writes them to 
# data.pickle file. The file is over-written.

# num_vpts = 3
# all_n, all_lines, all_centers = [], [], []

# for i in range(num_vpts):
#     print('Getting vanishing point {}'.format(i))
#     fig = plt.figure(); ax = fig.gca()
    
#     # Get at least three lines from user input
#     n_i, lines_i, centers_i = get_input_lines(im)
#     all_n.append(n_i)
#     all_lines.append(lines_i)
#     all_centers.append(centers_i)

# with open('data.pickle', 'wb') as f:
#     pickle.dump([all_n, all_lines, all_centers], f)

In [9]:
num_vpts = 3
all_n = [3, 3, 3]
all_lines = [np.array([[ 6.00984184e+01,  4.39180750e+01,  5.08525079e+01],
       [ 3.55967555e+02,  2.61196972e+02,  2.38082196e+02],
       [-4.43063561e+05, -3.03320106e+05, -3.57634299e+05]]), np.array([[-8.78361499e+01, -2.38082196e+02, -7.85902394e+01],
       [-1.84918210e+01, -8.32131947e+01, -1.15573882e+01],
       [ 1.52155917e+05,  5.22630321e+05,  1.21447527e+05]]), np.array([[  -97.08206047,   -94.77058284,  -143.31161307],
       [  161.80343411,   194.16412093,   175.67229989],
       [27282.92733661, 22302.86624441, 58494.79467975]])]
all_centers = [np.array([[6.33235896e+02, 1.19145774e+03, 1.28391685e+03],
       [1.13776404e+03, 9.60936005e+02, 1.22791167e+03],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00]]), np.array([[1.58325320e+03, 1.90454859e+03, 1.43647437e+03],
       [7.07829205e+02, 8.31493258e+02, 7.40189892e+02],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00]]), np.array([[1.26426929e+03, 1.37059726e+03, 1.36366283e+03],
       [5.89943846e+02, 5.54115943e+02, 7.79485012e+02],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])]

In [12]:
# Part (a)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))

for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(all_lines[i])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')

In [13]:
# Part (b) Computing and plotting the horizon
# <YOUR CODE> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
print(horizon_line)
# # <YOUR CODE> Plot the ground horizon line
# fig = plt.figure(); ax = fig.gca()
# plot_horizon_line(vpts, im)#im,horizon_line)
# fig.savefig('Q3_horizon.pdf', bbox_inches='tight')

[-2.82025858e-02 -9.99602228e-01  2.44012312e+02]


In [33]:
# Part (c) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpts)
print(u, v, f, K)

988.162361304116 784.956622107543 -1184.74842687507 Matrix([[-1184.74842687507, 0, 988.162361304116], [0, -1184.74842687507, 784.956622107543], [0, 0, 1]])


In [53]:
# Part (d) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(K, vpts)
print(R)

[-0.02542678 -0.9012174   0.4326207 ] [0.22876814 0.41603107 0.88010414]
[-0.02542678 -0.9012174   0.4326207   0.22876814  0.41603107  0.88010414
 -0.97314882  0.12134804  0.1955915 ]


In [ ]:
# Part (e) Generating fronto-parallel warps. Compute the 
# appropriate rotation to transform the world coordinates
# such that the axis of projection becomes the world
# X, Y and Z axes respectively. Use this rotation to estimate
# a homography that will be used to compute the output view.
# Apply the homography to generate the 3 fronto-parallel
# views and save them.

Rt = get_rotation_matrix_rectification()
H = get_homography()
    
fig = plt.figure(); ax = fig.gca()
# cv2.imwrite('Q3_im{:d}.jpg'.format(i), im_dst,
#             [int(cv2.IMWRITE_JPEG_QUALITY), 90])